# PySpark Implementation
In this notebook, we will implement train a NB model using the training set & preprocess the data with our heuristics.

### Imports

In [19]:
import csv, os, sys
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, StringIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from nltk.stem.snowball import SnowballStemmer
from pyspark.sql.types import ArrayType, StringType

In [20]:
# Initialize a spark session.
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [21]:
spark = init_spark()

### Converting training data into RDD

In [22]:
filename_train = "../dataset/train.csv"
filename_test = "../dataset/valid.csv"

train_rdd = spark.read.csv(filename_train, header=True, multiLine=True, inferSchema=True, escape='"', quote='"')
test_rdd = spark.read.csv(filename_test, header=True, multiLine=True, inferSchema=True, escape='"', quote='"')
train_rdd.show(5)

+--------+--------------------+--------------------+--------------------+-------------------+--------+
|      Id|               Title|                Body|                Tags|       CreationDate|       Y|
+--------+--------------------+--------------------+--------------------+-------------------+--------+
|34552656|Java: Repeat Task...|<p>I'm already fa...|      <java><repeat>|2016-01-01 00:21:59|LQ_CLOSE|
|34553034|Why are Java Opti...|<p>I'd like to un...|    <java><optional>|2016-01-01 02:03:20|      HQ|
|34553174|Text Overlay Imag...|<p>I am attemptin...|<javascript><imag...|2016-01-01 02:48:24|      HQ|
|34553318|Why ternary opera...|<p>The question i...|<swift><operators...|2016-01-01 03:30:17|      HQ|
|34553755|hide/show fab wit...|<p>I'm using cust...|<android><materia...|2016-01-01 05:21:48|      HQ|
+--------+--------------------+--------------------+--------------------+-------------------+--------+
only showing top 5 rows



## Preprocessing data
In this part, we will preprocess our data with three different heuristics. The first using tokenization, second using stemming, third removing stop words.

### Build training & testing dataframes

In [28]:
training = train_rdd.rdd \
    .map(lambda x: (x["Title"]+" "+x["Body"], x["Y"])) \
    .toDF(["Question", "Output"]) # change to collect(), limit is not a collect!!!!

testing = test_rdd.rdd \
    .map(lambda x: (x["Title"]+" "+x["Body"], x["Y"])) \
    .toDF(["Question", "Output"]) # change to collect()

### Heuristics

In [29]:
# HEURISTIC 1 - Tokenize the words
regexTokenizer = RegexTokenizer(inputCol="Question", outputCol="words", pattern="\\W")

# HEURISTIC 2 - Remove the stopwords
add_stopwords = ["the", "a", "be", "of", "and", "to", "why"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# HEURISTIC 3 - Stem words
# Use this library (TODO) - https://github.com/master/spark-stemming

In [30]:
# Build B.o.W model 
countVectors_h1 = CountVectorizer(inputCol="words", outputCol="features")
indexed_features_h1 = StringIndexer(inputCol = "Output", outputCol = "label")

countVectors_h2 = CountVectorizer(inputCol="filtered", outputCol="features")
indexed_features_h2 = StringIndexer(inputCol = "Output", outputCol = "label")


### Pipeline

In [31]:
# Pipeline with Heurisitc 1
pipeline_h1 = Pipeline(stages=[regexTokenizer, countVectors_h1, indexed_features_h1])

# Pipeline with Heurisitc 2
pipeline_h2 = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors_h2, indexed_features_h2])

# Pipeline with Heurisitc 3
# pipeline_h3 = Pipeline(stages=[regexTokenizer, stopwordsRemover, stemRemover, countVectors, label_stringIdx])
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))

In [32]:
pipeline_h1_fit = pipeline_h1.fit(training)
data_h1_train = pipeline_h1_fit.transform(training) #what does the transform do?
data_h1_train.show()

# pipeline_h2_fit = pipeline_h2.fit(training)
# data_h2 = pipeline_h2_fit.transform(training)
# data_h2.show()

# pipeline_h3_fit = pipeline_h3.fit(training)
# data_h3 = pipeline_h3_fit.transform(training)
# data_h3.show()

+--------------------+--------+--------------------+--------------------+-----+
|            Question|  Output|               words|            features|label|
+--------------------+--------+--------------------+--------------------+-----+
|Java: Repeat Task...|LQ_CLOSE|[java, repeat, ta...|(201488,[0,1,2,3,...|  1.0|
|Why are Java Opti...|      HQ|[why, are, java, ...|(201488,[0,1,4,7,...|  0.0|
|Text Overlay Imag...|      HQ|[text, overlay, i...|(201488,[0,1,2,3,...|  0.0|
|Why ternary opera...|      HQ|[why, ternary, op...|(201488,[0,1,2,3,...|  0.0|
|hide/show fab wit...|      HQ|[hide, show, fab,...|(201488,[0,1,4,5,...|  0.0|
|Accessing pointer...|LQ_CLOSE|[accessing, point...|(201488,[0,1,2,3,...|  1.0|
|How To Disable 2n...| LQ_EDIT|[how, to, disable...|(201488,[1,4,8,15...|  2.0|
|Resizing containe...| LQ_EDIT|[resizing, contai...|(201488,[1,2,3,4,...|  2.0|
|Changing Theme in...|      HQ|[changing, theme,...|(201488,[0,1,2,3,...|  0.0|
|TextBox Value Dis...| LQ_EDIT|[textbox,

In [ ]:
df_stemmed = data_h1_train.withColumn("words_stemmed", stemmer_udf("words_clean")).select('id', 'words_stemmed')

## Model training
In this part, we will train various pipelines through a NB classifier. 
- Hypertune the parameters
- Train the model
- Get the performance of the model

### Hypertuning parameters
Generally speaking, NB models don't require a lot of hypertuning, but, it remains good practice nonetheless.

In [25]:
# Split training data into train/validation set.
train_h1, validate_h1 = data_h1_train.randomSplit([0.7, 0.3], seed = 1234)

# Create grid to find best smoothing
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
paramGrid = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]).build()
cvEvaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")

# Cross-validate all smoothing values
cv = CrossValidator(estimator=nb, estimatorParamMaps=paramGrid, evaluator=cvEvaluator)
cvModel = cv.fit(train_h1)

# Make predictions on validation set
cvPredictions = cvModel.transform(validate_h1)

# Evaluate best model
evaluator.evaluate(cvPredictions)

NameError: name 'evaluator' is not defined

### Training best model